# 목차
<hr/>

<ol>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;">구글 드라이브 mount하여 연동하기</i></li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;">필요한 모듈 불러오기</i></li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 1</span> : 데이터 읽어오기</i></li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 2</span> : 데이터 정제</i></li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 3</span> : 데이터를 Tensor로 변환하기</i></li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 4</span> : 평가 데이터셋 분리</i>
        <ul style = "margin-top:10px; list-style:none;">
            <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">1. </span> 생성된 텐서를 소스와 타겟으로 분리하기</li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">2. </span> 훈련 데이터와 평가 데이터를 분리하기</li>
        </ul>
    </li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 5</span> : 인공지능 만들기</i>
        <ul style = "margin-top:10px; list-style:none;">
            <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">1. </span> loss 지정하기</li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">2. </span> 텍스트 생성 클래스 선언하기</li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">3. </span> Model Build</li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">4. </span> Model 훈련하기</li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">5. </span> 작문을 통해 모델 평가해보기</li>
        </ul>
    </li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;">회고</i>
</ol>

<div style = "height: 50px;"><div/>

# 구글 드라이브 mount하여 연동하기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<div style = "height: 50px;"><div/>

# 필요한 모듈 불러오기

In [4]:
import warnings
warnings.filterwarnings("ignore")

import glob
import os, re

import tensorflow        as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

<div style = "height: 50px;"><div/>

# <span style = "font-weight:bold;color:#0172d4;">STEP 1</span> : 데이터 읽어오기

In [11]:
txt_file_path = "/content/drive/MyDrive/data/*"
txt_list = glob.glob(txt_file_path, recursive = False)
print(txt_list[0])
print(type(txt_list))

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("\n데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

/content/drive/MyDrive/data/beatles.txt
<class 'list'>

데이터 크기: 187088
Examples:
 ['Yesterday, all my troubles seemed so far away', "Now it looks as though they're here to stay", "Oh, I believe in yesterday Suddenly, I'm not half the man I used to be"]


<div style = "height: 50px;"><div/>

# <span style = "font-weight:bold;color:#0172d4;">STEP 2</span> : 데이터 정제

<p style= "font-size:15px; line-height:24px;"> - preprocess_sentence() 사용하여 데이터 정제하기</p>

### 함수

In [12]:
# 문장 필터링하기
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

<div style = "height: 25px;"></div>

### 데이터 정제하기

In [13]:
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: 
        continue
    
    if sentence[-1] == ":": 
        continue
    
    # 토큰의 개수 15개 초과 시, 삭제
    if len(sentence.split(' ')) >17:
        continue
        
    # 정제를 하고 담아주세요tokenizer.num_words
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

<div style = "height: 25px;"></div>

### 정제된 데이터 확인하기

In [14]:
# 정제된 결과를 10개만 확인
corpus[:10]

['<start> yesterday , all my troubles seemed so far away <end>',
 '<start> now it looks as though they re here to stay <end>',
 '<start> oh , i believe in yesterday suddenly , i m not half the man i used to be <end>',
 '<start> there s a shadow hanging over me . <end>',
 '<start> oh , yesterday came suddenly why she had to go i don t know she wouldn t say <end>',
 '<start> now i need a place to hide away <end>',
 '<start> oh , i believe in yesterday why she had to go i don t know she wouldn t say <end>',
 '<start> now i need a place to hide away <end>',
 '<start> oh , i believe in yesterday <end>',
 '<start> speaking words of wisdom , let it be <end>']

<div style = "height: 50px;"><div/>

# <span style = "font-weight:bold;color:#0172d4;">STEP 3</span> : 데이터를 Tensor로 변환하기

### 함수

In [15]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=12000, 
    filters=' ',
    oov_token="<unk>")
    
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

<div style = "height: 25px;"></div>

### Tensor 변환하기

In [16]:
tensor, tokenizer = tokenize(corpus)

[[  2 888   4 ...   0   0   0]
 [  2  50  11 ...   0   0   0]
 [  2  41   4 ...   0   0   0]
 ...
 [  2 231   1 ...   0   0   0]
 [  2  10 501 ...   0   0   0]
 [  2 126  20 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f4450e5f750>


In [17]:
print(tensor[:3, :10])

[[   2  888    4   25   13 2583 2021   30  356  137]
 [   2   50   11  776   81  322   43   53   93   10]
 [   2   41    4    5  213   14  888 2316    4    5]]


<div style = "height: 50px;"><div/>

# <span style = "font-weight:bold;color:#0172d4;">STEP 4</span> : 평가 데이터셋 분리

## 1. 생성된 텐서를 소스와 타겟으로 분리하기
<hr/>

In [18]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
src_input = tensor[:, :-1] 

# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2  888    4   25   13 2583 2021   30  356  137    3    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]
[ 888    4   25   13 2583 2021   30  356  137    3    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


<div style = "height: 25px;"></div>

### 분리된 데이터 확인해보기

In [19]:
src_input

array([[  2, 888,   4, ...,   0,   0,   0],
       [  2,  50,  11, ...,   0,   0,   0],
       [  2,  41,   4, ...,   0,   0,   0],
       ...,
       [  2, 231,   1, ...,   0,   0,   0],
       [  2,  10, 501, ...,   0,   0,   0],
       [  2, 126,  20, ...,   0,   0,   0]], dtype=int32)

<div style = "height: 50px;"><div/>

## 2. 훈련 데이터와 평가 데이터를 분리하기
<hr/>

### train_test_split() 함수 사용하여 훈련 데이터와 검증 데이터 분리하기

In [20]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, random_state = 42)

<div style = "height: 25px;"></div>

### from_tensor_slices() 메소드를 이용해 훈련 데이터 생성하기

In [21]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

In [22]:
# from_tensor_slices() 메소드를 이용해 tf.data.Dataset객체를 생성
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 32), dtype=tf.int32, name=None), TensorSpec(shape=(256, 32), dtype=tf.int32, name=None))>

<div style = "height: 50px;"><div/>

# <span style = "font-weight:bold;color:#0172d4;">STEP 5</span> : 인공지능 만들기

## 1. loss 지정하기
<hr/>

In [23]:
#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

<div style = "height: 50px;"><div/>

## 2. 텍스트 생성 클래스 선언하기
<hr/>

In [24]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

<div style = "height: 50px;"><div/>

## 3. Model Build
<hr/>

In [25]:
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [26]:
model.summary()

ValueError: ignored

<p style= "font-size:15px; line-height:24px;">모델이 빌드되지 않은 상태에서 summary를 진행했기 때문에, 모델이 무슨 일을 하는 모델인지 알려주지 못한다. 따라서, 모델을 build하기 위해서는 어떤 데이터를 입력받아야하는지 알려줘야 하므로 데이터를(소스 데이터 = 특성값 = x값)을 모델에 넣어야한다.</p>

<div style = "height: 25px;"><div/>

### 데이터를 모델에 넣어주기

In [27]:
<p style= "font-size:15px; line-height:24px;">모델이 빌드되지 않은 상태에서 summary를 진행했기 때문에, 모델이 무슨 일을 하는 모델인지 알려주지 못한다. 따라서, 모델을 build하기 위해서는 어떤 데이터를 입력받아야하는지 알려줘야 하므로 데이터를(소스 데이터 = 특성값 = x값)을 모델에 넣어야한다.</p>

<div style = "height: 25px;"><div/>

### 데이터를 모델에 넣어주기for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

<tf.Tensor: shape=(256, 32, 12001), dtype=float32, numpy=
array([[[ 2.27355864e-04,  8.57987397e-05,  8.42949084e-05, ...,
          4.07271524e-04,  1.49766100e-04, -5.81003369e-05],
        [ 2.20872520e-04,  1.59759365e-05, -1.37129218e-05, ...,
          4.17082891e-04,  1.19816192e-04, -8.72928795e-05],
        [ 4.34078247e-04, -9.07997819e-05, -3.21990054e-04, ...,
          5.20239351e-04, -2.94688594e-04, -2.74416292e-04],
        ...,
        [ 3.56825884e-03,  2.75751832e-03,  1.23283477e-04, ...,
         -1.38996949e-03,  1.51385757e-04,  1.35969964e-03],
        [ 3.59382806e-03,  2.77332682e-03,  1.33903537e-04, ...,
         -1.40085595e-03,  1.67417311e-04,  1.35435676e-03],
        [ 3.61532182e-03,  2.78613600e-03,  1.44411970e-04, ...,
         -1.40862446e-03,  1.83653945e-04,  1.34916173e-03]],

       [[ 2.27355864e-04,  8.57987397e-05,  8.42949084e-05, ...,
          4.07271524e-04,  1.49766100e-04, -5.81003369e-05],
        [ 4.22416982e-04, -9.81055709e-05, -4

<div style = "height: 25px;"><div/>

### 모델의 정보 확인하기

In [28]:
# 이제 모델이 뭐하는지 알았으니 summary해서 모델의 정보 확인하기
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  3072256   
                                                                 
 lstm (LSTM)                 multiple                  5246976   
                                                                 
 lstm_1 (LSTM)               multiple                  8392704   
                                                                 
 dense (Dense)               multiple                  12301025  
                                                                 
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


<p style= "font-size:15px; line-height:24px;">모델의 파라미터가 29,012,961이 나왔다.<br/>
따라서, 모델을 돌리는데에 있어 <span style = 'background-color:#fff5b1; padding:0.2px;'>시간이 오래 걸릴 것</span>이라고 예상된다.<br/>
뿐만아니라, 파라미터가 많기 때문에 <span style = 'background-color:#fff5b1; padding:0.2px;'>과적합</span>이 발생할 수 있다는 문제가 발생할 수 있다고 생각한다.</p>

<div style = "height: 50px;"><div/>

## 4. Model 훈련하기
<hr/>

In [29]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True, reduction = 'none')

model.compile(loss = loss, optimizer = optimizer)
model.fit(dataset, validation_data = (enc_val, dec_val), epochs = 10)

Epoch 1/10
535/535 [==============================] - 121s 222ms/step - loss: 1.7351 - val_loss: 1.5025
Epoch 2/10
535/535 [==============================] - 118s 220ms/step - loss: 1.4493 - val_loss: 1.4212
Epoch 3/10
535/535 [==============================] - 118s 220ms/step - loss: 1.3718 - val_loss: 1.3655
Epoch 4/10
535/535 [==============================] - 118s 220ms/step - loss: 1.3105 - val_loss: 1.3225
Epoch 5/10
535/535 [==============================] - 118s 220ms/step - loss: 1.2583 - val_loss: 1.2916
Epoch 6/10
535/535 [==============================] - 118s 220ms/step - loss: 1.2117 - val_loss: 1.2653
Epoch 7/10
535/535 [==============================] - 118s 220ms/step - loss: 1.1696 - val_loss: 1.2437
Epoch 8/10
535/535 [==============================] - 118s 220ms/step - loss: 1.1303 - val_loss: 1.2259
Epoch 9/10
535/535 [==============================] - 118s 220ms/step - loss: 1.0937 - val_loss: 1.2103
Epoch 10/10
535/535 [==============================] - 118s 220m

<p style= "font-size:15px; line-height:24px;">모델을 훈련시킨 시간은 <span style = 'background-color:#fff5b1; padding:0.2px;'>총 2시간 50분</span>이 걸렸다.<br/>
예상한대로, 히든레이어와, 임베딩 사이즈를 크게 주었기 때문에 파라미터가 많이 생성되었고, 이로 인해 모델의 훈련 속도가 느려진 것 같다.
loss값은 과적합 되었을 것이라는 예상과는 다르게, 훈련 데이터의 loss값과 검증 데이터셋의 loss값이 크게 차이가 나지 않으면서, <span style = 'background-color:#fff5b1; padding:0.2px;'>1.2 이하로 떨어졌다.</span></p>

<div style = "height: 50px;"><div/>

## 5. 작문을 통해 모델 평가해보기
<hr/>

### 작문 진행 함수 선언하기

In [30]:
# 작문 진행 함수

def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
        
    return generated

<div style = "height: 50px;"><div/>

### 작문해보기

In [31]:
generate_text(model, tokenizer, init_sentence="<start> always", max_len=20)

'<start> always since i m a <unk> <end> '

In [32]:
generate_text(model, tokenizer, init_sentence="<start> i think", max_len=20)

'<start> i think i m a bad bitch <end> '

In [34]:
generate_text(model, tokenizer, init_sentence="<start> i want", max_len=20)

'<start> i want to be a little selfish <end> '

In [35]:
generate_text(model, tokenizer, init_sentence="<start> please", max_len=20)

'<start> please don t stop the music , i m a flirt <end> '

In [36]:
generate_text(model, tokenizer, init_sentence="<start> from", max_len=20)

'<start> from the bottom of the night <end> '

In [37]:
generate_text(model, tokenizer, init_sentence="<start> love", max_len=20)

'<start> love is a little selfish <end> '

In [38]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you <end> '

In [39]:
generate_text(model, tokenizer, init_sentence="<start> believe", max_len=20)

'<start> believe me , love me , love me , love me , love me , love me , love '

<p style= "font-size:15px; line-height:24px;">대부분 잘 나온 것을 확인할 수 있었다. 그런데 도서관 사서를 많이 사랑하나보다.<br/> 취향이 확고한 모델인 것 같다.</p>

<div style = "height: 50px;"><div/>

# 회고
<hr/>

<p style= "font-size:16px; line-height:30px;">이번 프로젝트를 하면서 처음으로 코랩 pro를 사용해보았는데, 평소 사용하던 jupyter notebook과는 사용 방법이 많이 달라 사용하는데 애를 많이 먹었습니다.</p>

<div style = "height: 50px;"></div>

## 1. 코랩에서 파일을 불러오기 위해서는 구글 드라이브에 올려 불러오는 것이 편하다.
<hr/>

<p style= "font-size:16px; line-height:30px;">로컬 파일을 불러오고자 경로 설정을 하였는데 되지 않아 확인해본 결과,</p>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

<p style= "font-size:16px; line-height:30px;">를 통해 구글 드라이브와 연동을 시킨 후 파일을 불러올 수 있었다.<br/>
또한, 이때 기본 주소(HOME)는 content/가 되며, 이는 os.getcwd()를 통해 현재 경로를 확인할 수 있다.</p>

<div style = "height: 50px;"></div>

## 2. 드라이브 상의 파일의 경로 확인하기
<hr/>

<img src = "img/colab_01.png">

<p style= "font-size:16px; line-height:30px;">왼쪽 파일 아이콘을 눌러 폴더 목록을 열은 후 원하는 파일에 오른쪽 클릭을 하여 경로를 복사할 수 있다.</p>

<div style = "height: 50px;"></div>

## 3. 모델을 fit할 때에는 꼭 변수를 지정하자
<hr/>

<p style= "font-size:16px; line-height:30px;">모델의 훈련 과정과 데이터의 loss값 등을 이용하기 위해서는 model.fit()할 때, </p>

In [ ]:
history = model.fit()

<p style= "font-size:16px; line-height:30px;">이렇게 변수를 지정하여 값을 저장해야, 후에 loss값 비교, 모델별 성능 비교 등을 할 수 있으므로, <span style = 'background-color:#fff5b1; padding:0.2px;'>반드시 변수를 생성하는 것이 좋다.</span> <br/>
모델을 모두 학습시키고 난 후, 이 사실을 알게되어서 매우 슬펐다. <br/>
이번에 모델을 돌릴 때에는 시간이 많이 소요됐으므로 넘어가고, 다음에는 반드시 변수에 저장하여 사용할 것이다.</p>

<div style = "height: 50px;"></div>

## 4. 서브 클래스를 사용하여 모델을 설계했을 때에는 tf.keras.callbacks.ModelCheckpoint로 저장하자
<hr/>

- [콜백 사용하는 방법 1](https://deep-deep-deep.tistory.com/53)
- [콜백 사용하는 방법 2](https://teddylee777.github.io/tensorflow/keras-%EC%BD%9C%EB%B0%B1%ED%95%A8%EC%88%98-vol-02)
- [텐서플로우 콜백](https://www.tensorflow.org/guide/checkpoint?hl=ko)
- [model.save()](https://www.tensorflow.org/guide/keras/save_and_serialize?hl=ko)

<p style= "font-size:16px; line-height:30px;">파일을 닫고 다시 열었을 때에는 모델을 다시 사용할 수 없으므로, model.save()를 사용하여 .h5(h5파일)로 저장해야한다. 이 후, tf.saved_model.load()로 쉽게 불러올 수 있다.<br/>
<span style = 'background-color:#fff5b1; padding:0.2px;'>그러나, 이 방법은 함수형 또는 시퀀셜 API(모델을 순차적으로 쌓음)를 사용하여 만든 모델에만 사용하 가능하다.</span></p>

<p style= "font-size:16px; line-height:30px;">실제로 모델을 돌렸을 시,  save_format='tf'으로 지정하라는 오류가 발생하는 것을 확인할 수 있었다.</p>

<p style= "font-size:16px; line-height:30px;">이 후, 모델을 tf로 저장하여 불러오니, model.summary()는 먹히나, 모델 입력 값의 shape가 다르다는 오류가 발생하였다.
확인해본 결과, model.save()는 서브클래스를 사용한 모델은 저장할 수 없으며, 가중치만 저장하여 사용할 수 있다는 것을 확인할 수 있었다. <br/>
때문에, 파일로 저장된 모델과 실제 적용하는 모델이 달라져서 적용할 수 없었다.</p>

<p style= "font-size:16px; line-height:30px;">서브 클래스를 사용하여 모델을 설계했다면, <span style = 'background-color:#fff5b1; padding:0.2px;'>tf.keras.callbacks.ModelCheckpoint</span>로 모델의 저장하여 불러올 수 있다.<br/>
ModelCheckpoint는 모델이 학습하면서 정의한 조건(fit())을 만족했을 때, Model의 가중치 값을 중간중간 저장한다. 학습시간이 오래 걸린다면 loss값이 올라갔을 때에는 저장하지 않고, 떨어졌을 때에는 가중치를 저장함으로서, 중간에 과적합이 발생하였거나, 다양한 문제가 발생했을 때, 다시 중간의 가중치 값을 불러와서 학습을 이어나갈 수 있다.</p>

<div style = "height: 50px;"></div>

## 5. 코랩은 모델을 fit()하는 도중, 시간이 지나면 런타임이 끊긴다.  
<hr/>

<p style= "font-size:16px; line-height:30px;">이번에는 모델의 훈련 시간이 오래 걸리기 때문에, 자기 직전 model.fit()을 실행시켰다.
자고 일어나면 모델 훈련이 끝났을 것이라는 기대와는 달리, 중간에 런타임이 끊겨 모델을 다시 사용할 수도 없고, 훈련 도중에 런타임이 끊긴 것을 확인할 수 있었다.<br/>
조심하여 돌려야겠다.</p>